In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import os
import sys
sys.path.insert(0, '../../src/')
from data_loader import load_data, load_game_data
pd.set_option('display.max_columns', None)

* * *

### Data preprocessing

In [3]:
# concatenating game-data.csv and lineups-all-seasons.csv

path_lineups = "/home/matej/Documents/Projects/nba-data-mining/data/processed/lineups-all-seasons.csv"
path_game_data = "/home/matej/Documents/Projects/nba-data-mining/data/processed/game-data-extended.csv"
path_players = "/home/matej/Documents/Projects/nba-data-mining/data/raw/player-data/player_info.csv"

lineups = pd.read_csv(path_lineups)
game_data = pd.read_csv(path_game_data)
players = pd.read_csv(path_players)

# merged = pd.concat([game_data, lineups], axis=1)
# merged.to_csv("game-data-extended.csv", index=False)

In [ ]:
# sorting players that started the game by their ID

""" ALREADY SORTED
for i, row in lineups.iterrows():

    game_data.at[i, "home_player1_id":"home_player5_id"] =  np.asarray(game_data.iloc[i]["home_player1_id" : "home_player5_id"].sort_values(ascending=True))
    game_data.at[i, "visitor_player1_id":"visitor_player5_id"] =  np.asarray(game_data.iloc[i]["visitor_player1_id" : "visitor_player5_id"].sort_values(ascending=True))

    lineups.at[i, "home_player1_id":"home_player5_id"] =  np.asarray(lineups.iloc[i]["home_player1_id" : "home_player5_id"].sort_values(ascending=True))
    lineups.at[i, "visitor_player1_id":"visitor_player5_id"] =  np.asarray(lineups.iloc[i]["visitor_player1_id" : "visitor_player5_id"].sort_values(ascending=True))
"""

In [5]:
lineups_home = lineups[["season_name", "home_team_id", "home_player1_id", "home_player2_id", "home_player3_id", "home_player4_id", "home_player5_id"]].drop_duplicates()
lineups_visitor = lineups[["season_name", "visitor_team_id", "visitor_player1_id", "visitor_player2_id", "visitor_player3_id", "visitor_player4_id", "visitor_player5_id"]].drop_duplicates()

lineups_home.columns = range(lineups_home.shape[1])
lineups_visitor.columns = range(lineups_visitor.shape[1])

all_lineups = pd.concat([lineups_home, lineups_visitor])
all_lineups.rename(columns = {0:'season', 1:'team_id', 2:"player1_id", 3:"player2_id", 4:"player3_id", 5:"player4_id", 6:"player5_id"}, inplace = True)
all_lineups.drop_duplicates(inplace=True)

In [ ]:
# TODO TODO TODO TODO

lineups_seasons = all_lineups.groupby(["season", "team_id"]).size().unstack()
lineups_seasons.rename(index={"2000-01":2000, "2001-02":2001, "2002-03":2002, "2003-04":2003, "2004-05":2004, "2005-06":2005, "2006-07":2006, "2007-08":2007, "2008-09":2008, "2009-10":2009, "2010-11":2010, "2011-12":2011, "2012-13":2012, "2013-14":2013, "2014-15":2014, "2015-16":2015, "2016-17":2016, "2017-18":2017, "2018-19":2018}, inplace=True)
lineups_seasons

In [7]:
tmp = all_lineups.groupby(["season"]).size() / all_lineups.groupby(["season"])["team_id"].nunique()
tmp.rename(index={"2000-01":2000, "2001-02":2001, "2002-03":2002, "2003-04":2003, "2004-05":2004, "2005-06":2005, "2006-07":2006, "2007-08":2007, "2008-09":2008, "2009-10":2009, "2010-11":2010, "2011-12":2011, "2012-13":2012, "2013-14":2013, "2014-15":2014, "2015-16":2015, "2016-17":2016, "2017-18":2017, "2018-19":2018}, inplace=True)

def custom_legend_name(new_names):
    for i, new_name in enumerate(new_names):
        fig.data[i].name = new_name

In [8]:
fig = px.line(tmp, width=1400, title="Number of average starting lineups per season")
custom_legend_name(['lineups-per-season'])
fig.update_yaxes(range=[0,30])
fig.show()

In [9]:
fig = px.line(lineups_seasons, width=1400, title="Number of different starting lineups per team per season")
fig.show()

* * *

### Findings 1
- There were 22033 different starting lineups in 22965 games played in all seasons together.
- Dirk Nowitzki started most games (started 1304 games).

In [10]:
lineups_home = game_data[["home_player1_id", "home_player2_id", "home_player3_id", "home_player4_id", "home_player5_id"]].drop_duplicates()
lineups_visitor = game_data[["visitor_player1_id", "visitor_player2_id", "visitor_player3_id", "visitor_player4_id", "visitor_player5_id"]].drop_duplicates()

lineups_home.columns = range(lineups_home.shape[1])
lineups_visitor.columns = range(lineups_visitor.shape[1])

all_lineups = pd.concat([lineups_home, lineups_visitor]).drop_duplicates()

In [11]:
all_lineups.mode()

,0,1,2,3,4
0,1717.0,1717.0,1891.0,1891.0,201596.0


In [12]:
players.groupby("Player ID")["games_started"].sum().sort_values(ascending=False).iloc[0]

1304

In [13]:
players[players["Player ID"] == players.groupby("Player ID")["games_started"].sum().sort_values(ascending=False).index[0]]["Player"].unique()

array(['Dirk Nowitzki'], dtype=object)

* * *

### Findings LeBron James
- Draft number = 1
- Min games played = 51 (seasons 2011 & 2018)
- Min games sarted = 55 (season 2018)
- Max games played = 82 (season 2017)
- Max games started = 79 (season 2004)
- Mean games played = 74.875
- Mean games started = 67.3125

### Findings Tony Parker
- Draft number = 28
- Min games played = season ()
- Min games sarted = season ()
- Max games played = season ()
- Max games started = season ()
- Mean games played = 74.875
- Mean games started = 

### Findings Steph Curry
- Draft number = 7
- Min games played = season ()
- Min games sarted = season ()
- Max games played = season ()
- Max games started = season ()
- Mean games played = 74.875
- Mean games started = 

In [15]:
lebron_james = players[(players["Player"] == "LeBron James")].groupby(["Season"])[["games_played", "games_started"]].sum()
tony_parker = players[(players["Player"] == "Tony Parker")].groupby(["Season"])[["games_played", "games_started"]].sum()
steph_curry = players[(players["Player"] == "Stephen Curry")].groupby(["Season"])[["games_played", "games_started"]].sum()
kawhi_leonard = players[(players["Player"] == "Kawhi Leonard")].groupby(["Season"])[["games_played", "games_started"]].sum()

In [16]:
lebron_james.rename(index={"2000-01":2000, "2001-02":2001, "2002-03":2002, "2003-04":2003, "2004-05":2004, "2005-06":2005, "2006-07":2006, "2007-08":2007, "2008-09":2008, "2009-10":2009, "2010-11":2010, "2011-12":2011, "2012-13":2012, "2013-14":2013, "2014-15":2014, "2015-16":2015, "2016-17":2016, "2017-18":2017, "2018-19":2018}, inplace=True)
tony_parker.rename(index={"2000-01":2000, "2001-02":2001, "2002-03":2002, "2003-04":2003, "2004-05":2004, "2005-06":2005, "2006-07":2006, "2007-08":2007, "2008-09":2008, "2009-10":2009, "2010-11":2010, "2011-12":2011, "2012-13":2012, "2013-14":2013, "2014-15":2014, "2015-16":2015, "2016-17":2016, "2017-18":2017, "2018-19":2018}, inplace=True)
steph_curry.rename(index={"2000-01":2000, "2001-02":2001, "2002-03":2002, "2003-04":2003, "2004-05":2004, "2005-06":2005, "2006-07":2006, "2007-08":2007, "2008-09":2008, "2009-10":2009, "2010-11":2010, "2011-12":2011, "2012-13":2012, "2013-14":2013, "2014-15":2014, "2015-16":2015, "2016-17":2016, "2017-18":2017, "2018-19":2018}, inplace=True)
kawhi_leonard.rename(index={"2000-01":2000, "2001-02":2001, "2002-03":2002, "2003-04":2003, "2004-05":2004, "2005-06":2005, "2006-07":2006, "2007-08":2007, "2008-09":2008, "2009-10":2009, "2010-11":2010, "2011-12":2011, "2012-13":2012, "2013-14":2013, "2014-15":2014, "2015-16":2015, "2016-17":2016, "2017-18":2017, "2018-19":2018}, inplace=True)

In [17]:
lebron_james.agg(["min", "max", "mean"])

,games_played,games_started
min,0.000000,0.000000
max,82.000000,79.000000
mean,70.470588,63.352941


In [18]:
tony_parker.agg(["min", "max", "mean"])

,games_played,games_started
min,55.000000,0.000000
max,82.000000,81.000000
mean,69.666667,60.055556


In [19]:
steph_curry.agg(["min", "max", "mean"])

,games_played,games_started
min,0.000000,0.000000
max,80.000000,77.000000
mean,63.090909,56.454545


In [20]:
kawhi_leonard.agg(["min", "max", "mean"])

,games_played,games_started
min,0.000000,0.000000
max,74.000000,71.000000
mean,51.888889,46.888889


In [21]:
fig = px.line(lebron_james, markers=True, title='Comparison of started and played games - LeBron James')
fig.add_annotation(x=2014, y=62, text="Knee, lower back (11)", showarrow=False, arrowhead=4)
fig.add_annotation(x=2018, y=46, text="Strained left groin (25)", showarrow=False, arrowhead=4)
fig.update_yaxes(range=[0,83])
fig.show()

In [22]:
fig = px.line(tony_parker, markers=True, title='Comparison of started and played games - Tony Parker')
fig.add_annotation(x=2007, y=57, text="Sprained left index finger (5)", showarrow=False, arrowhead=4)
fig.add_annotation(x=2017, y=30, text="Raptured quadriceps", showarrow=False, arrowhead=4)
fig.update_yaxes(range=[0,83])
fig.show()

In [23]:
fig = px.line(steph_curry, markers=True, title='Comparison of started and played games - Steph Curry')
fig.add_annotation(x=2011, y=20, text="Ankle surgery", showarrow=False, arrowhead=4)
fig.add_annotation(x=2017, y=30, text="Sprained ankle", showarrow=False, arrowhead=4)
fig.update_yaxes(range=[0,83])
fig.show()

In [24]:
fig = px.line(kawhi_leonard, markers=True, title='Comparison of started and played games - Kawhi Leonard')
fig.add_annotation(x=2012, y=46, text="Quad, knee injury (18)", showarrow=False, arrowhead=4)
fig.add_annotation(x=2014, y=52, text="Metacarpal fracture (17)", showarrow=False, arrowhead=4)
fig.add_annotation(x=2017, y=4, text="Sore quadriceps (27)", showarrow=False, arrowhead=4)
fig.update_yaxes(range=[0,83])
fig.show()